In [ ]:
import polars as pl
import datetime
import random
import time
today_date = datetime.date.today()
import yaml
from ai_knowledge.ai_knowledge.base.convert import *

today_date = datetime.date.today()

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [ ]:
start = time.time()

In [ ]:
import os
output_dir = f'data_backup/output_feature_nadir_multiprogression_{today_date}/'
if output_dir.split('/')[1] not in os.listdir('data_backup/'):
    os.makedirs(output_dir)

In [ ]:
#DB Connection Details
host = "c0-eurekarwd-web.crdektqhzze9.us-east-1.redshift.amazonaws.com"
port = "5439"
dbname = "c0_eurekarwd"
user = ""
password = ""

In [ ]:
schema = 'qcca_cdm_master'
# schema = 'dts_cdm_master'
clip_year = 2019
# clip_year = 2003

In [ ]:
db_conn_uri = "redshift://{user}:{pwd}@{server}:{port}/{db}".format(
                    user=user.replace('@','%40'), 
                    pwd=password.replace('@','%40'), 
                    server=host, 
                    port=port, 
                    db=dbname)

In [ ]:
with open('./ai_knowledge/ai_knowledge/base/keys_labs.yaml','r') as file:
    lab_kb = yaml.safe_load(file)
    
lab_kb = {c['key']:c for c in lab_kb}

with open('./ai_knowledge/ai_knowledge/base/units.yaml','r') as file:
    unite_conv_map = yaml.safe_load(file)
    
unite_conv_map = {c['base']:c for c in unite_conv_map}

In [ ]:
def convert_unit(df):
    lab_test = df['key'].unique().drop_nulls()
    for key in lab_test:
        if key not in lab_kb.keys():
            raise KeyError(f'Lab test {key} not registered')
        std_unit = lab_kb[key]['attributes']['units']
        unit_keys = list(unite_conv_map[std_unit]['convert'].keys())
        df = df.with_columns(
                pl.struct(['test_value_numeric','test_unit_standard_name','key'])
                          .apply(lambda x: eval(unite_conv_map[std_unit]['convert'][x['test_unit_standard_name']].split('.')[-1])(x['test_value_numeric'])\
                                 if (x['key']==key) & (x['test_unit_standard_name'] in (unit_keys))\
                                 else x['test_value_numeric']                                                      
                                ).alias('test_value_numeric')
        )
        df = df.with_columns(
                        pl.when((pl.col('key')==key) & ((pl.col('test_unit_standard_name')==std_unit) | (pl.col('test_unit_standard_name').is_in(unite_conv_map[std_unit]['convert'].keys()))))
                .then(pl.lit(std_unit))
                .when(pl.col('key')==key)
                .then(None)
                .otherwise(pl.col('test_unit_standard_name'))
                .alias('test_unit_standard_name'))\
        .drop_nulls('test_unit_standard_name')
    return df

In [ ]:
# def rule_check(test_values,thresold):
#     criteria_flag = [None]
#     for i in range(1,len(test_values)):
#         flag=False
#         if ((test_values[i]>=thresold) and (test_values[i-1]>=thresold) and ((test_values[i]-test_values[i-1])>=thresold)):
#             flag =True
#         if ((test_values[i]>=thresold) and (test_values[i-1]>=thresold) and (((test_values[i]-test_values[i-1])/(test_values[i-1]+0.0001))>=0.25)):
#             flag=True
#         criteria_flag.append(flag)
        
#     return criteria_flag

# def rule_check(test_values,thresold):
#     criteria_flag = [None]
#     min_val = test_values[0]
#     for i in range(1,len(test_values)):
#         if min_val>test_values[i]:
#             min_val=test_values[i]
#         flag=False
#         if (((test_values[i]-min_val)>=thresold) and (((test_values[i]-min_val)/(min_val+0.0001))>=0.25)):
#             flag =True
#         criteria_flag.append(flag)
        
#     return criteria_flag


def rule_check(test_values,thresold):
    criteria_flag = [None]
    min_val = test_values[0]
    for i in range(1,len(test_values)):
        if min_val>test_values[i]:
            min_val=test_values[i]
        flag=False
        if (((test_values[i]-min_val)>=thresold) and (((test_values[i]-min_val)/(min_val+0.0001))>=0.25)):
            if ((len(test_values)>(i+1)) and ((test_values[i+1]-min_val)>=thresold) and (((test_values[i+1]-min_val)/(min_val+0.0001))>=0.25)):
                flag =True
        criteria_flag.append(flag)
        
    return criteria_flag


In [ ]:
chai_patid_query = "select chai_patient_id,diagnosis_date,diagnosis_code_standard_code,diagnosis_code_standard_name from {schema}.condition where (lower(diagnosis_code_standard_name) like '%multiple%myeloma%' or lower(diagnosis_code_standard_code) like 'C90.0%') and extract(year from diagnosis_date) > {clip_year}".format(schema=schema,clip_year=clip_year)
pat_id_df = pl.read_database(chai_patid_query,db_conn_uri)
print(pat_id_df.shape)
pat_id_df_updated = pat_id_df\
        .select(['chai_patient_id','diagnosis_date'])\
        .sort(by =['chai_patient_id','diagnosis_date'],descending=False)\
        .unique(subset=['chai_patient_id'],keep='first')\
        .drop_nulls()\
        .rename({'diagnosis_date':'clip_date'})\
        .with_columns(pl.col('clip_date').cast(pl.Date))
print(pat_id_df_updated.shape)
pat_ids = ','.join(["'"+x+"'" for x in pat_id_df_updated.select(['chai_patient_id']).unique().to_series().to_list()])

In [ ]:
def get_next_progression(clip_date_df,first):
   
    ## Serum Test Result

    #Serum Test results on above `pat_ids`
    pat_serum_test_query = "select chai_patient_id,source_labid,test_date,test_name_standard_code,test_name_standard_name,test_value_numeric,test_unit_standard_name from {schema}.patient_test where (chai_patient_id in ({pat_ids})) and (test_name_standard_code in ('33358-3','51435-6','35559-4','94400-9','50796-2')) and (extract(year from test_date) > {clip_year})".format(pat_ids=pat_ids,schema=schema,clip_year=clip_year)

    #Load the query to polars Dataframe
    pat_serum_test_df = pl.read_database(pat_serum_test_query,db_conn_uri)
        
    #Cast `test_value_numeric` to float (from str)
    pat_serum_test_df = pat_serum_test_df.with_columns(
        pl.col('test_value_numeric').cast(pl.Float64, strict=False),
        pl.col('test_date').cast(pl.Date)
    )
    
    if first:
        pat_serum_test_df = pat_serum_test_df.join(clip_date_df,on='chai_patient_id',how='left')\
                                        .filter(pl.col('test_date')>=pl.col('clip_date'))
    else:
        pat_serum_test_df = pat_serum_test_df.join(clip_date_df,on='chai_patient_id',how='left')\
                                        .filter(pl.col('test_date')>pl.col('clip_date'))
    
    pat_serum_test_df = pat_serum_test_df.with_columns(pl.lit('m_protein_in_serum').alias('key'))
    #Drop Nulls and Filter units '%' 
    pat_serum_test_df = pat_serum_test_df.drop_nulls(subset=['test_value_numeric', 'test_unit_standard_name'])\
            .filter(pl.col('test_unit_standard_name') != "%")
    pat_serum_test_df = convert_unit(pat_serum_test_df)
    
    pat_serum_test_df = pat_serum_test_df.groupby(by=['chai_patient_id','test_date','test_name_standard_code']).agg(pl.col('test_value_numeric').mean())\
                                     .select(['chai_patient_id','test_date','test_value_numeric','test_name_standard_code'])

    
    pat_serum_test_df = pat_serum_test_df.sort(['chai_patient_id','test_name_standard_code','test_date'],descending=False)\
                                    .groupby(['chai_patient_id','test_name_standard_code']).agg(pl.col('test_value_numeric'),pl.col('test_date'))\
                                    .filter(pl.col('test_value_numeric').apply(lambda x: len(x)>1))
    
    pat_serum_test_df = pat_serum_test_df.with_columns(pl.col('test_value_numeric').apply(lambda x:rule_check(x,0.5)).alias('criteria_flag'))\
                                    .explode(['test_value_numeric','test_date','criteria_flag'])
    
    pat_serum_test_df = pat_serum_test_df.filter(pl.col('criteria_flag')==True)\
                                    .sort(['chai_patient_id','test_date'],descending=False)\
                                    .unique(subset='chai_patient_id',keep='first')
    
    
    ## Urine Test Results
    pat_urine_test_query = "select chai_patient_id,source_labid,test_date,test_name_standard_code,test_name_standard_name,test_value_numeric,test_unit_standard_name from {schema}.patient_test where (chai_patient_id in ({pat_ids})) and (test_name_standard_code in ('42482-0','35560-2')) and (extract(year from test_date) > {clip_year})".format(pat_ids=pat_ids,schema=schema,clip_year=clip_year)

    pat_urine_test_df = pl.read_database(pat_urine_test_query,db_conn_uri)
    
    #Cast `test_value_numeric` to float64, drop nulls and filter units '%'
    pat_urine_test_df = pat_urine_test_df.with_columns(
                pl.col('test_value_numeric').cast(pl.Float64, strict=False),
                pl.col('test_date').cast(pl.Date)
            ).drop_nulls(subset=['test_value_numeric', 'test_unit_standard_name'])\
            .filter(pl.col('test_unit_standard_name') != "%")
    
    if first:
        pat_urine_test_df = pat_urine_test_df.join(clip_date_df,on='chai_patient_id',how='left')\
                                        .filter(pl.col('test_date')>=pl.col('clip_date'))
    else:
        pat_urine_test_df = pat_urine_test_df.join(clip_date_df,on='chai_patient_id',how='left')\
                                        .filter(pl.col('test_date')>pl.col('clip_date'))
    
    pat_urine_test_df = pat_urine_test_df.with_columns(pl.lit('m_protein_in_urine').alias('key'))
    pat_urine_test_df = convert_unit(pat_urine_test_df)
    
    pat_urine_test_df = pat_urine_test_df.groupby(by=['chai_patient_id','test_date','test_name_standard_code']).agg(pl.col('test_value_numeric').mean())\
                                     .select(['chai_patient_id','test_date','test_value_numeric','test_name_standard_code'])
    
    pat_urine_test_df =  pat_urine_test_df.sort(['chai_patient_id','test_name_standard_code','test_date'],descending=False)\
                                    .groupby(['chai_patient_id','test_name_standard_code']).agg(pl.col('test_value_numeric'),pl.col('test_date'))\
                                    .filter(pl.col('test_value_numeric').apply(lambda x: len(x)>1))
    
    
    pat_urine_test_df = pat_urine_test_df.with_columns(pl.col('test_value_numeric').apply(lambda x:rule_check(x,200)).alias('criteria_flag'))\
                                    .explode(['test_value_numeric','test_date','criteria_flag'])
    
    pat_urine_test_df = pat_urine_test_df.filter(pl.col('criteria_flag')==True)\
                                    .sort(['chai_patient_id','test_date'],descending=False)\
                                    .unique(subset='chai_patient_id',keep='first')

    rule1_df = pat_serum_test_df.join(pat_urine_test_df,on='chai_patient_id',how='outer',suffix='_urine')\
                                .with_columns(
                                    pl.min_horizontal('test_date','test_date_urine')\
                                        .alias('min_date')
                                )
    
    criteria_df = pl.DataFrame(clip_date_df['chai_patient_id'].unique())

    criteria_df = criteria_df.join(rule1_df.select(['chai_patient_id','min_date']),on='chai_patient_id',how='left')\
                            .with_columns(
                                pl.col('min_date').is_not_null()\
                                .alias('criteria_1')
                            )
    
    ## Criteria 2
    ## Free light chain
    
    #Free ligh Test results on above `pat_ids`
    pat_free_light_query = "select chai_patient_id,source_labid,test_date,test_name_standard_code,test_name_standard_name,test_value_numeric,test_unit_standard_name from {schema}.patient_test where (chai_patient_id in ({pat_ids})) and (test_name_standard_code in ('36916-5','33944-0')) and (extract(year from test_date) > {clip_year})".format(pat_ids=pat_ids,schema=schema,clip_year=clip_year)
    
    #Load the query to polars Dataframe
    pat_free_light_df = pl.read_database(pat_free_light_query,db_conn_uri)
    
    pat_free_light_df = pat_free_light_df.drop_nulls(subset=['test_value_numeric'])\
                                    .with_columns(
                                        pl.col('test_value_numeric').apply(lambda x: x.replace('<','').replace('>','').replace(',','.')).cast(pl.Float64, strict=False),
                                        pl.col('test_date').cast(pl.Date)
                                    )\
                                    .drop_nulls(subset=['test_value_numeric'])
    
    if first:
        pat_free_light_df = pat_free_light_df.join(clip_date_df,on='chai_patient_id',how='left')\
                                        .filter(pl.col('test_date')>=pl.col('clip_date'))
    else:
        pat_free_light_df = pat_free_light_df.join(clip_date_df,on='chai_patient_id',how='left')\
                                        .filter(pl.col('test_date')>pl.col('clip_date'))
    
    pat_free_light_df = pat_free_light_df.with_columns(pl.lit('serum_free_light').alias('key'))
    
    pat_free_light_df = convert_unit(pat_free_light_df)
    
    pat_free_light_df = pat_free_light_df.groupby(by=['chai_patient_id','test_date','test_name_standard_code']).agg(pl.col('test_value_numeric').mean())\
                                     .select(['chai_patient_id','test_date','test_value_numeric','test_name_standard_code'])


    pat_free_light_df = pat_free_light_df.sort(['chai_patient_id','test_name_standard_code','test_date'],descending=False)\
                                    .groupby(['chai_patient_id','test_name_standard_code']).agg(pl.col('test_value_numeric'),pl.col('test_date'))\
                                    .filter(pl.col('test_value_numeric').apply(lambda x: len(x)>1))
    
    
    pat_free_light_df = pat_free_light_df.with_columns(pl.col('test_value_numeric').apply(lambda x:rule_check(x,10)).alias('criteria_flag'))\
                                    .explode(['test_value_numeric','test_date','criteria_flag'])
    
    pat_free_light_df = pat_free_light_df.filter(pl.col('criteria_flag')==True)\
                                    .sort(['chai_patient_id','test_date'],descending=False)\
                                    .unique(subset='chai_patient_id',keep='first')\
                                    .select(['chai_patient_id','test_date'])
    
    criteria_df = criteria_df.join(pat_free_light_df,on='chai_patient_id',how='left')\
                        .with_columns(
                            pl.min_horizontal('min_date','test_date')\
                            .alias('min_date'),
                            pl.col('test_date').is_not_null()\
                            .alias('criteria_2')
                        ).drop('test_date')
    
    ## Criteria 4
    ## Plasmacytoma 
    #Plasmacytoma diagnosis on above `pat_ids`
    con_plasmacytoma_query = "select chai_patient_id,diagnosis_date,diagnosis_code_standard_code,diagnosis_code_standard_name from {schema}.condition where (chai_patient_id in ({pat_ids})) and ((lower(diagnosis_code_standard_name) like '%extramedullary%plasmacytoma%') or (lower(diagnosis_code_standard_code) like 'C90.2%')) and (extract(year from diagnosis_date) > {clip_year})".format(pat_ids=pat_ids,schema=schema,clip_year=clip_year)
    #Load the query to polars Dataframe
    con_plasmacytoma_df = pl.read_database(con_plasmacytoma_query,db_conn_uri)
    
    con_plasmacytoma_df = con_plasmacytoma_df.sort(by=['chai_patient_id','diagnosis_date'],descending=False)\
                                        .unique(subset='chai_patient_id',keep='first')\
                                        .with_columns(
                                            pl.col('diagnosis_date').cast(pl.Date)
                                        ).select(['chai_patient_id','diagnosis_date'])
    
    if first:
        con_plasmacytoma_df = con_plasmacytoma_df.join(clip_date_df,on='chai_patient_id',how='left')\
                                        .filter(pl.col('diagnosis_date')>=pl.col('clip_date'))
    else:
        con_plasmacytoma_df = con_plasmacytoma_df.join(clip_date_df,on='chai_patient_id',how='left')\
                                        .filter(pl.col('diagnosis_date')>pl.col('clip_date'))
    
    criteria_df = criteria_df.join(con_plasmacytoma_df,on='chai_patient_id',how='left')\
                        .with_columns(
                            pl.min_horizontal('min_date','diagnosis_date')\
                            .alias('min_date'),
                            pl.col('diagnosis_date').is_not_null()\
                            .alias('criteria_4')
                        ).drop('diagnosis_date')
    
    criteria_df = criteria_df.with_columns(
        (pl.col('criteria_1') | pl.col('criteria_2') | pl.col('criteria_4'))\
        .alias('final_selection')
    )

    criteria_df = criteria_df.filter(pl.col('final_selection')==True)\
                            .select(['chai_patient_id','min_date','criteria_1'])
    return  criteria_df

In [ ]:
flag = 1
final_criteria_df = []
first=True
c=0
while flag==1:
    print(c)
    criteria_df = get_next_progression(pat_id_df_updated,first)
    first=False
    if criteria_df.shape[0]>0:
        final_criteria_df.append(criteria_df)
        pat_id_df_updated = pat_id_df_updated.join(criteria_df,on='chai_patient_id',how='left')\
                                            .with_columns(
                                                pl.when(pl.col('min_date').is_null())\
                                                .then(pl.col('clip_date'))\
                                                .otherwise(pl.col('min_date'))\
                                                .alias('clip_date')
                                            )\
                                            .drop(['min_date','criteria_1'])
    else:
        flag = 0
    c+=1

final_criteria_df = pl.concat(final_criteria_df)       

In [ ]:
final_criteria_df.head()

In [ ]:
final_criteria_df.shape

In [ ]:
final_criteria_df.unique().shape

In [ ]:
final_criteria_df.write_csv(f'{output_dir}/criteria_multiprogression_label_{today_date}.csv')